Here we evaluate the performance of our application. Interestingly, we can use LLMs themselves to gauge how our RAG pipeline is performing.

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
import datetime

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatGooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores.pinecone import Pinecone
from langchain.evaluation.qa import QAGenerateChain, QAEvalChain

### Setting up

In [4]:
palm_key = os.getenv('PALM_API_KEY')

In [7]:
loader = CSVLoader("data/sample.csv")
data = loader.load()

In [10]:
llm = ChatGooglePalm(google_api_key=palm_key, temperature=0.0)
embeddings = GooglePalmEmbeddings(google_api_key=palm_key)

In [20]:
import os
import pinecone

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY_02"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [22]:
docsearch = Pinecone.from_documents(data, embeddings, index_name='langchain-demo')

### Retrieval

In [42]:

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(), 
    verbose=True,
)

### QAGenerateChain
Automates the creation of question answer sets for evaluatiom.

In [24]:
llm = ChatGooglePalm(google_api_key=palm_key)

In [25]:
# LLM-Generated examples
example_gen_chain = QAGenerateChain.from_llm(llm=llm)

# the warning below can be safely ignored
examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [33]:
examples

[{'qa_pairs': {'query': "What is John Doe's email address?",
   'answer': 'john.doe@email.com'}},
 {'qa_pairs': {'query': "What is Jane Smith's email address?",
   'answer': 'jane.smith@email.com'}},
 {'qa_pairs': {'query': "What is Bob Johnson's email address?",
   'answer': 'bob.johnson@email.com'}},
 {'qa_pairs': {'query': "What is Alice Williams's email address?",
   'answer': 'alice.williams@email.com'}},
 {'qa_pairs': {'query': "What is Charlie Brown's email address?",
   'answer': 'charlie.brown@email.com'}}]

### Evaluation

In [43]:
qa.run(examples[0]['qa_pairs']['answer'])



> Entering new RetrievalQA chain...

> Finished chain.


'John Doe is a 28-year-old man who lives in New York City. His phone number is 555-1234.'

In [ ]:

# Manual Evaluation
import langchain
langchain.debug = True

qa.run(examples[0]["query"])

# Turn off the debug mode
langchain.debug = False

# LLM assisted evaluation
predictions = qa.apply(examples)

llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

graded_outputs[0]

# LangChain evaluation platform
# The LangChain evaluation platform, LangChain Plus, can be accessed here https://www.langchain.plus/.
# Use the invite code `lang_learners_2023`
# Reminder: Download your notebook to your local computer to save your work.
